In [ ]:
from openai import OpenAI

def generate_refined_test_code(formatted_code: str, compiler_error_message: str) -> str:
    client = OpenAI(api_key="not-telling-you")

    prompt = f"""
**Java Test Fixer {{

@persona {{
You are a senior Java developer and a test engineering expert.
You are given a test file that **fails to compile** due to syntax errors. 
Your job is to correct the file so it compiles successfully, using the compiler error messages for guidance.
}}

@instruction {{
- Fix the Java test code provided using the compiler's error output.
- You may add missing import statements, fix syntax, correct types, or close braces — do **whatever is required** to make the code compile.
- Do not change the intended logic or structure unless the error forces a correction.
- If the test is incomplete or malformed, make reasonable corrections while staying true to the original logic.
- Always output a complete and standalone Java file.

@rules {{
@rule1: Output must be pure Java source code only — no Markdown, no explanation, and no comments.
@rule2: Include the full test class from package declaration to closing brace.
@rule3: Use only JUnit 4 APIs.
@rule4: Do not modify or remove test logic unless necessary to fix a compile error.
@rule5: If you must insert additional classes or mock data for the test to compile, keep it minimal and relevant.
@rule6: If the compilation error can be resolved by deleting a line, a method, or a class, do so!
}}

@input {{
###broken_test_code
{formatted_code}

###compiler_error_output
{compiler_error_message}
}}

@output {{
###fixed_test_code
}}
}}
}}"""

    response = client.chat.completions.create(
        model="gpt-4.1",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        max_tokens=4000
    )

    return response.choices[0].message.content


In [12]:
import subprocess
import os

def compile_java_test(test_file):
    project_root = "/Users/xuanantran/Desktop/comp4130/asg3/An_Tran_u7542145/lang_1_buggy"

    # Path to the Maven repository and relevant .jar files
    junit_jar = "/Users/xuanantran/.m2/repository/junit/junit/4.11/junit-4.11.jar"
    hamcrest_jar = "/Users/xuanantran/.m2/repository/org/hamcrest/hamcrest-core/1.3/hamcrest-core-1.3.jar"
    compiled_classes_dir = os.path.join(project_root, "target/classes")

    # Output directory for test .class files (to avoid polluting the source folder)
    test_output_dir = os.path.join(project_root, "target/test-classes")
    os.makedirs(test_output_dir, exist_ok=True)

    # Full classpath including compiled classes and dependencies
    classpath = f"{compiled_classes_dir}:{junit_jar}:{hamcrest_jar}"

    # Construct javac command
    command = [
        "javac",
        "-cp", classpath,
        "-d", test_output_dir,
        test_file
    ]

    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        # Return empty string if compilation is successful
        return result.stdout
    except subprocess.CalledProcessError as e:
        # If compilation fails, return the error message
        return e.stderr

In [13]:
def feedback_loop(original_code, error_msg, absolute_path):
    max_attempts = 3
    attempt = 0
    code = original_code

    while attempt < max_attempts:
        print(f"[INFO] Attempt {attempt + 1} to fix test...")

        # Generate refined code using the OpenAI API
        refined_code = generate_refined_test_code(code, error_msg)
        print("[INFO] Generated refined code:\n", refined_code)

        # Save and test the revised code
        with open(absolute_path, "w") as f:
            f.write(refined_code)

        result = compile_java_test(absolute_path)

        if result.strip() == "":
            print("[SUCCESS] Test compiles after refinement.")
            return refined_code  # Success

        # Otherwise, retry
        code = refined_code
        error_msg = result
        attempt += 1

    print("[FAIL] All refinement attempts failed.")
    return None  # All attempts failed


In [14]:
import os
import pandas as pd

# === Config
csv_path = "/Users/xuanantran/Desktop/comp4130/asg3/An_Tran_u7542145/lang_1_buggy/Test_Data_Task4-2.csv"
project_base_dir = "/Users/xuanantran/Desktop/comp4130/asg3/An_Tran_u7542145/lang_1_buggy"

# === Load data
df = pd.read_csv(csv_path)

# === Prepare column for results
df["Runnable Test Code"] = ""

# === Main loop
for index, row in df.iterrows():
    formatted_code = row["Code After Formatting"]
    saved_path = row["Saved Path"]
    
    # === Build absolute save path
    absolute_path = os.path.join(project_base_dir, saved_path)
    os.makedirs(os.path.dirname(absolute_path), exist_ok=True)
    with open(absolute_path, "w") as f:
        f.write(formatted_code)

    # === Try compiling
    result = compile_java_test(absolute_path)

    if result.strip() == "":
        df.at[index, "Runnable Test Code"] = formatted_code
    else:
        print(f"[FAIL] Compilation failed at row {index}. Invoking feedback loop...")
        refined_code = feedback_loop(formatted_code, result, absolute_path)
        if refined_code:
            df.at[index, "Runnable Test Code"] = refined_code

csv_path_out = "/Users/xuanantran/Desktop/comp4130/asg3/An_Tran_u7542145/lang_1_buggy/Test_Data_Task5.csv"
df.to_csv(csv_path_out, index=False)

[FAIL] Compilation failed at row 14. Invoking feedback loop...
[INFO] Attempt 1 to fix test...
[INFO] Generated refined code:
 package org.apache.commons.lang3;

import java.util.Iterator;
import java.util.NoSuchElementException;
import org.junit.Test;
import static org.junit.Assert.*;

public class CharRange_iterator__Test {

    // Helper: Use reflection to access the private constructor of CharRange
    private CharRange createCharRange(char start, char end, boolean negated) {
        try {
            java.lang.reflect.Constructor<CharRange> ctor = CharRange.class.getDeclaredConstructor(char.class, char.class, boolean.class);
            ctor.setAccessible(true);
            return ctor.newInstance(start, end, negated);
        } catch (Exception e) {
            throw new RuntimeException("Failed to create CharRange instance via reflection", e);
        }
    }

    @Test
    public void testIterator_NormalRange() {
        // Range: 'a' to 'e', not negated
        CharRange range